# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_test = pd.read_csv('data/y_test.csv')
y_val = pd.read_csv('data/y_val.csv')

In [3]:
X_train_numeric.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,540.0,0.0,130.0,0.0,130.0,0.0,0.0,0.0,10.0,2006.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,622.0,0.0,45.0,0.0,0.0,0.0,0.0,0.0,6.0,2009.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,534.0,171.0,159.0,0.0,0.0,0.0,0.0,0.0,9.0,2008.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,608.0,237.0,152.0,0.0,0.0,0.0,0.0,0.0,3.0,2010.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,812.0,168.0,52.0,0.0,0.0,0.0,0.0,0.0,4.0,2009.0


In [4]:
X_test_cat.head()

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_Pave,Alley_missing,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [5]:
# Combine all features
X_train = pd.concat([X_train_cat, X_train_numeric], axis = 1, sort = False)
X_val = pd.concat([X_val_cat, X_val_numeric], axis = 1, sort = False)
X_test = pd.concat([X_test_cat, X_test_numeric], axis = 1, sort = False)

As a refresher, preview the training data: 

In [6]:
# Preview the data
X_train.head()

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_Pave,Alley_missing,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,540.0,0.0,130.0,0.0,130.0,0.0,0.0,0.0,10.0,2006.0
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,622.0,0.0,45.0,0.0,0.0,0.0,0.0,0.0,6.0,2009.0
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,534.0,171.0,159.0,0.0,0.0,0.0,0.0,0.0,9.0,2008.0
3,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,608.0,237.0,152.0,0.0,0.0,0.0,0.0,0.0,3.0,2010.0
4,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,812.0,168.0,52.0,0.0,0.0,0.0,0.0,0.0,4.0,2009.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

Here, we're calling .shape on our training data so that we can use the result as n_features, so we know how big to make our input layer.

In [7]:
# How big input layer?
n_features = (X_train.shape[1],)
print(n_features)

(296,)


Create your baseline model. Yo will notice is exihibits strange behavior.

*Note:* When you run this model or other models later on, you may get a comment from tf letting you about optimizing your GPU

In [8]:
# Baseline model
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(units = 100, activation = 'relu', input_shape = (n_features)))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(units = 50, activation = 'relu'))

# Output layer
baseline_model.add(layers.Dense(units = 1, activation = 'linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 7ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

Epoch 137/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 947us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=========

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [12]:
# Numeric column names
numeric_columns = X_train_numeric.columns 

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(data = ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(data = ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(data = ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [13]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 
Again, you may get some strange behavior.

In [14]:
# Train the model
normalized_input_model.fit(X_train, y_train, epochs=150, batch_size = 32, validation_data = (X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=========================

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [15]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [16]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, y_train_scaled, epochs=150, batch_size = 32, validation_data = (X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 6ms/step - loss: 0.4050 - mse: 0.4050 - val_loss: 36093091840.0000 - val_mse: 36093091840.0000
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2143 - mse: 0.2143 - val_loss: 36093046784.0000 - val_mse: 36093046784.0000
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1806 - mse: 0.1806 - val_loss: 36093059072.0000 - val_mse: 36093059072.0000
Epoch 4/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1490 - mse: 0.1490 - val_loss: 36093042688.0000 - val_mse: 36093042688.0000
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1335 - mse: 0.1335 - val_loss: 36093038592.0000 - val_mse: 36093038592.0000
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1178 - mse: 0.1178 - val_loss: 36093050880.0000 - val_mse: 36093050880.0000
Epoch 7/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1117 - mse: 0.1117 

Epoch 55/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0239 - mse: 0.0239 - val_loss: 36093059072.0000 - val_mse: 36093059072.0000
Epoch 56/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0239 - mse: 0.0239 - val_loss: 36093059072.0000 - val_mse: 36093059072.0000
Epoch 57/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0230 - mse: 0.0230 - val_loss: 36093046784.0000 - val_mse: 36093046784.0000
Epoch 58/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0225 - mse: 0.0225 - val_loss: 36093059072.0000 - val_mse: 36093059072.0000
Epoch 59/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0223 - mse: 0.0223 - val_loss: 36093059072.0000 - val_mse: 36093059072.0000
Epoch 60/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0218 - mse: 0.0218 - val_loss: 36093046784.0000 - val_mse: 36093046784.0000
Epoch 61/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0214 - mse: 

Epoch 109/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0103 - mse: 0.0103 - val_loss: 36093059072.0000 - val_mse: 36093059072.0000
Epoch 110/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0102 - mse: 0.0102 - val_loss: 36093059072.0000 - val_mse: 36093059072.0000
Epoch 111/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0101 - mse: 0.0101 - val_loss: 36093063168.0000 - val_mse: 36093063168.0000
Epoch 112/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0100 - mse: 0.0100 - val_loss: 36093059072.0000 - val_mse: 36093059072.0000
Epoch 113/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0098 - mse: 0.0098 - val_loss: 36093067264.0000 - val_mse: 36093067264.0000
Epoch 114/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0099 - mse: 0.0099 - val_loss: 36093067264.0000 - val_mse: 36093067264.0000
Epoch 115/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0098 

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [17]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 2ms/step - loss: 0.0058 - mse: 0.0058


[0.005811183247715235, 0.005811183247715235]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [18]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 1ms/step - loss: 0.1446 - mse: 0.1446


[0.14456704258918762, 0.14456704258918762]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [19]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

29879.008354249272

Great. Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'` 
  
[Documentation on the He Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/HeNormal)

In [22]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(units = 100, activation = 'relu', input_shape = (n_features), kernel_initializer = 'he_normal'))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])
# Train the model
he_model.fit(X_train, 
                   y_train_scaled, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.4084 - mse: 0.4084 - val_loss: 0.1888 - val_mse: 0.1888
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2458 - mse: 0.2458 - val_loss: 0.1681 - val_mse: 0.1681
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1990 - mse: 0.1990 - val_loss: 0.1487 - val_mse: 0.1487
Epoch 4/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1711 - mse: 0.1711 - val_loss: 0.1349 - val_mse: 0.1349
Epoch 5/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1533 - mse: 0.1533 - val_loss: 0.1303 - val_mse: 0.1303
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1318 - mse: 0.1318 - val_loss: 0.1238 - val_mse: 0.1238
Epoch 7/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1300 - mse: 0.1300 - val_loss: 0.1279 - val_mse: 0.1279
Epoch 8/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1165 - m

33/33 [==============================] - 0s 1ms/step - loss: 0.0257 - mse: 0.0257 - val_loss: 0.1166 - val_mse: 0.1166
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0256 - mse: 0.0256 - val_loss: 0.1165 - val_mse: 0.1165
Epoch 65/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0257 - mse: 0.0257 - val_loss: 0.1166 - val_mse: 0.1166
Epoch 66/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0250 - mse: 0.0250 - val_loss: 0.1173 - val_mse: 0.1173
Epoch 67/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0250 - mse: 0.0250 - val_loss: 0.1186 - val_mse: 0.1186
Epoch 68/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0244 - mse: 0.0244 - val_loss: 0.1204 - val_mse: 0.1204
Epoch 69/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0241 - mse: 0.0241 - val_loss: 0.1174 - val_mse: 0.1174
Epoch 70/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0237 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0133 - mse: 0.0133 - val_loss: 0.1221 - val_mse: 0.1221
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0135 - mse: 0.0135 - val_loss: 0.1216 - val_mse: 0.1216
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0131 - mse: 0.0131 - val_loss: 0.1224 - val_mse: 0.1224
Epoch 128/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0128 - mse: 0.0128 - val_loss: 0.1217 - val_mse: 0.1217
Epoch 129/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0129 - mse: 0.0129 - val_loss: 0.1232 - val_mse: 0.1232
Epoch 130/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0128 - mse: 0.0128 - val_loss: 0.1230 - val_mse: 0.1230
Epoch 131/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0125 - mse: 0.0125 - val_loss: 0.1228 - val_mse: 0.1228
Epoch 132/150
33/33 [==============================] - 0s 1ms/step - 

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [23]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 651us/step - loss: 0.0105 - mse: 0.0105


[0.010470193810760975, 0.010470193810760975]

Evaluate the model (`he_model`) on validate data (`X_val` and `y_val_scaled`)

In [24]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 989us/step - loss: 0.1228 - mse: 0.1228


[0.12282108515501022, 0.12282108515501022]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 
  
[Documentation on the Lecun Normal Initializer](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/LecunNormal)

In [28]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, kernel_initializer='lecun_normal', activation='relu', input_shape=n_features))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 5ms/step - loss: 0.4650 - mse: 0.4650 - val_loss: 0.1681 - val_mse: 0.1681
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2325 - mse: 0.2325 - val_loss: 0.1437 - val_mse: 0.1437
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: 0.2089 - mse: 0.2089 - val_loss: 0.1574 - val_mse: 0.1574
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1750 - mse: 0.1750 - val_loss: 0.1328 - val_mse: 0.1328
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1574 - mse: 0.1574 - val_loss: 0.1273 - val_mse: 0.1273
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1394 - mse: 0.1394 - val_loss: 0.1272 - val_mse: 0.1272
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1323 - mse: 0.1323 - val_loss: 0.1309 - val_mse: 0.1309
Epoch 8/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1227 - m

33/33 [==============================] - 0s 1ms/step - loss: 0.0259 - mse: 0.0259 - val_loss: 0.1343 - val_mse: 0.1343
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0260 - mse: 0.0260 - val_loss: 0.1315 - val_mse: 0.1315
Epoch 65/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0251 - mse: 0.0251 - val_loss: 0.1337 - val_mse: 0.1337
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0244 - mse: 0.0244 - val_loss: 0.1302 - val_mse: 0.1302
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0239 - mse: 0.0239 - val_loss: 0.1299 - val_mse: 0.1299
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0240 - mse: 0.0240 - val_loss: 0.1321 - val_mse: 0.1321
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0238 - mse: 0.0238 - val_loss: 0.1333 - val_mse: 0.1333
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0227 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0112 - mse: 0.0112 - val_loss: 0.1290 - val_mse: 0.1290
Epoch 126/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0110 - mse: 0.0110 - val_loss: 0.1298 - val_mse: 0.1298
Epoch 127/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0108 - mse: 0.0108 - val_loss: 0.1308 - val_mse: 0.1308
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0109 - mse: 0.0109 - val_loss: 0.1304 - val_mse: 0.1304
Epoch 129/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0105 - mse: 0.0105 - val_loss: 0.1303 - val_mse: 0.1303
Epoch 130/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0105 - mse: 0.0105 - val_loss: 0.1302 - val_mse: 0.1302
Epoch 131/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0105 - mse: 0.0105 - val_loss: 0.1299 - val_mse: 0.1299
Epoch 132/150
33/33 [==============================] - 0s 1ms/step - 

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [26]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 1ms/step - loss: 0.0070 - mse: 0.0070


[0.007043883670121431, 0.007043883670121431]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [27]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 900us/step - loss: 0.1256 - mse: 0.1256


[0.12557673454284668, 0.12557673454284668]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

[Documentation on the RMS Prop Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/experimental/RMSprop)

In [30]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
# Code here
rmsprop_model.compile(optimizer='rmsprop', 
                 loss='mse', 
                 metrics=['mse'])
# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 1s 18ms/step - loss: 0.3267 - mse: 0.3267 - val_loss: 0.1352 - val_mse: 0.1352
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1694 - mse: 0.1694 - val_loss: 0.1207 - val_mse: 0.1207
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1165 - mse: 0.1165 - val_loss: 0.1340 - val_mse: 0.1340
Epoch 4/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1072 - mse: 0.1072 - val_loss: 0.1010 - val_mse: 0.1010
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0912 - mse: 0.0912 - val_loss: 0.1077 - val_mse: 0.1077
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0708 - mse: 0.0708 - val_loss: 0.1158 - val_mse: 0.1158
Epoch 7/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0564 - mse: 0.0564 - val_loss: 0.1072 - val_mse: 0.1072
Epoch 8/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0496 - 

33/33 [==============================] - 0s 2ms/step - loss: 0.0097 - mse: 0.0097 - val_loss: 0.1003 - val_mse: 0.1003
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0113 - mse: 0.0113 - val_loss: 0.1083 - val_mse: 0.1083
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0072 - mse: 0.0072 - val_loss: 0.1047 - val_mse: 0.1047
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0071 - mse: 0.0071 - val_loss: 0.1084 - val_mse: 0.1084
Epoch 67/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0100 - mse: 0.0100 - val_loss: 0.0971 - val_mse: 0.0971
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0118 - mse: 0.0118 - val_loss: 0.1104 - val_mse: 0.1104
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0063 - mse: 0.0063 - val_loss: 0.1076 - val_mse: 0.1076
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0122 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0048 - mse: 0.0048 - val_loss: 0.0932 - val_mse: 0.0932
Epoch 126/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0073 - mse: 0.0073 - val_loss: 0.0966 - val_mse: 0.0966
Epoch 127/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0043 - mse: 0.0043 - val_loss: 0.0963 - val_mse: 0.0963
Epoch 128/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0038 - mse: 0.0038 - val_loss: 0.0937 - val_mse: 0.0937
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0083 - mse: 0.0083 - val_loss: 0.0991 - val_mse: 0.0991
Epoch 130/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0042 - mse: 0.0042 - val_loss: 0.0934 - val_mse: 0.0934
Epoch 131/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0070 - mse: 0.0070 - val_loss: 0.0992 - val_mse: 0.0992
Epoch 132/150
33/33 [==============================] - 0s 2ms/step - 

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [31]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 817us/step - loss: 0.0055 - mse: 0.0055


[0.005527545232325792, 0.005527545232325792]

Evaluate the model (`rmsprop_model`) on training data (`X_val` and `y_val_scaled`) 

In [32]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 735us/step - loss: 0.0932 - mse: 0.0932


[0.09320317208766937, 0.09320317208766937]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric

[Documentation on the Adam Optimizer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam)

In [34]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=n_features))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
# Code here
adam_model.compile(optimizer = 'Adam',
                  loss = 'mse',
                  metrics = ['mse'])
# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.4196 - mse: 0.4196 - val_loss: 0.1297 - val_mse: 0.1297
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1683 - mse: 0.1683 - val_loss: 0.0878 - val_mse: 0.0878
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1173 - mse: 0.1173 - val_loss: 0.0904 - val_mse: 0.0904
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0929 - mse: 0.0929 - val_loss: 0.0876 - val_mse: 0.0876
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0711 - mse: 0.0711 - val_loss: 0.0865 - val_mse: 0.0865
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0537 - mse: 0.0537 - val_loss: 0.0894 - val_mse: 0.0894
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0424 - mse: 0.0424 - val_loss: 0.0857 - val_mse: 0.0857
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0314 - m

33/33 [==============================] - 0s 1ms/step - loss: 0.0210 - mse: 0.0210 - val_loss: 0.1112 - val_mse: 0.1112
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0136 - mse: 0.0136 - val_loss: 0.1187 - val_mse: 0.1187
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0112 - mse: 0.0112 - val_loss: 0.1052 - val_mse: 0.1052
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0055 - mse: 0.0055 - val_loss: 0.1100 - val_mse: 0.1100
Epoch 67/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0032 - mse: 0.0032 - val_loss: 0.1053 - val_mse: 0.1053
Epoch 68/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0023 - mse: 0.0023 - val_loss: 0.1041 - val_mse: 0.1041
Epoch 69/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0016 - mse: 0.0016 - val_loss: 0.1058 - val_mse: 0.1058
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0011 - mse: 0

Epoch 124/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0013 - mse: 0.0013 - val_loss: 0.1052 - val_mse: 0.1052
Epoch 125/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0018 - mse: 0.0018 - val_loss: 0.1009 - val_mse: 0.1009
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0016 - mse: 0.0016 - val_loss: 0.1076 - val_mse: 0.1076
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0017 - mse: 0.0017 - val_loss: 0.1015 - val_mse: 0.1015
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0021 - mse: 0.0021 - val_loss: 0.1064 - val_mse: 0.1064
Epoch 129/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0022 - mse: 0.0022 - val_loss: 0.0978 - val_mse: 0.0978
Epoch 130/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0024 - mse: 0.0024 - val_loss: 0.1013 - val_mse: 0.1013
Epoch 131/150
33/33 [==============================] - 0s 1ms/step - 

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [35]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 370us/step - loss: 0.0013 - mse: 0.0013


[0.0013398734154179692, 0.0013398734154179692]

Evaluate the model (`adam_model`) on training data (`X_val` and `y_val_scaled`) 

In [36]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 1ms/step - loss: 0.1051 - mse: 0.1051


[0.10512933880090714, 0.10512933880090714]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [37]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)

5/5 [==============================] - 0s 1ms/step - loss: 0.1537 - mse: 0.1537


[0.1536710262298584, 0.1536710262298584]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [38]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_val, y_val_pred))

29879.008354249272

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.